In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

/home/aiteam/miniconda3/envs/3dfm/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class CausalConv1d(nn.Conv1d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.causal_padding = self.dilation[0] * (self.kernel_size[0] -1)

    def forward(self, x):
        return self._conv_forward(F.pad(x, [self.causal_padding, 0]), self.weight, self.bias)

In [4]:
test = CausalConv1d(3, 10, kernel_size=4, dilation=1)

In [5]:
test.causal_padding

3

In [6]:
test.weight.shape

torch.Size([10, 3, 4])

In [7]:
test._conv_forward(torch.randn(7, 3, 300), test.weight, test.bias).shape

torch.Size([7, 10, 297])

In [8]:
F.pad(torch.randn(3, 10), [2,0])

tensor([[ 0.0000,  0.0000,  0.5056, -0.2466, -0.1164,  1.4550,  0.1818,  0.4643,
          0.7300, -0.6947, -0.1402,  0.0484],
        [ 0.0000,  0.0000,  0.5896,  0.0440,  1.0827,  0.2574, -1.4395,  3.6763,
         -0.2315,  0.0047, -0.4282, -2.1393],
        [ 0.0000,  0.0000, -0.1773,  0.3543, -0.9901,  1.6098, -1.2676, -2.8507,
          0.2691,  0.3769, -1.0275,  0.5519]])

In [14]:
class CausalConvTranspose1d(nn.ConvTranspose1d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.causal_padding = self.dilation[0] * (self.kernel_size[0] - 1) + self.output_padding[0] + 1 - self.stride[0]
    
    def forward(self, x, output_size=None):
        if self.padding_mode != 'zeros':
            raise ValueError('Only `zeros` padding mode is supported for ConvTranspose1d')

        assert isinstance(self.padding, tuple)
        output_padding = self._output_padding(
            x, output_size, self.stride, self.padding, self.kernel_size, self.dilation)
        return F.conv_transpose1d(
            x, self.weight, self.bias, self.stride, self.padding,
            output_padding, self.groups, self.dilation)[...,:-self.causal_padding]

In [15]:
test = CausalConvTranspose1d(1, 3, 3)
test.padding

(0,)

In [16]:
test(torch.randn(3, 10), output_size=20)

TypeError: can only concatenate tuple (not "int") to tuple

In [25]:
test = CausalConvTranspose1d(in_channels=2*3,
                               out_channels=3,
                               kernel_size=2*1, stride=1)

In [27]:
test(torch.randn(6, 10)).shape

torch.Size([3, 10])

In [17]:
class ResidualUnit(nn.Module):
    def __init__(self, in_channels, out_channels, dilation):
        super().__init__()
        
        self.dilation = dilation

        self.layers = nn.Sequential(
            CausalConv1d(in_channels=in_channels, out_channels=out_channels,
                      kernel_size=7, dilation=dilation),
            nn.ELU(),
            nn.Conv1d(in_channels=in_channels, out_channels=out_channels,
                      kernel_size=1)
        )

    def forward(self, x):
        return x + self.layers(x)

In [18]:
class DecoderBlock(nn.Module):
    def __init__(self, out_channels, stride):
        super().__init__()

        self.layers = nn.Sequential(
            CausalConvTranspose1d(in_channels=2*out_channels,
                               out_channels=out_channels,
                               kernel_size=2*stride, stride=stride),
            nn.ELU(),
            ResidualUnit(in_channels=out_channels, out_channels=out_channels,
                         dilation=1),
            nn.ELU(),
            ResidualUnit(in_channels=out_channels, out_channels=out_channels,
                         dilation=3),
            nn.ELU(),
            ResidualUnit(in_channels=out_channels, out_channels=out_channels,
                         dilation=9),

        )

    def forward(self, x):
        return self.layers(x)

In [19]:
class Decoder(nn.Module):
    def __init__(self, C, D):
        super().__init__()
        
        self.layers = nn.Sequential(
            CausalConv1d(in_channels=D, out_channels=16*C, kernel_size=7),
            nn.ELU(),
            DecoderBlock(out_channels=8*C, stride=8),
            nn.ELU(),
            DecoderBlock(out_channels=4*C, stride=5),
            nn.ELU(),
            DecoderBlock(out_channels=2*C, stride=4),
            nn.ELU(),
            DecoderBlock(out_channels=C, stride=2),
            nn.ELU(),
            CausalConv1d(in_channels=C, out_channels=1, kernel_size=7)
        )
    
    def forward(self, x):
        return self.layers(x)

In [20]:
decoder = Decoder(C=1, D=3)

In [24]:
decoder(torch.randn(1, 3, 100)).shape

torch.Size([1, 1, 32000])

In [ ]:
weight = torch.nn.Parameter(torch.Tensor(o))

In [28]:
def causal_padding(x, kernel):
    if kernel.shape[-1] % 2 == 0:
        kernel = F.pad(kernel, [1,0], value=0.0)

    x = F.pad(x, [kernel.shape[-1]-1, 0], value=0.0)
    return x, kernel

In [ ]:
def causal_conv(x, kernel, bias=None, **kwargs):
    x, kernel = causal_padding(x, kernel)
    return F.conv1d(x, kernel, bias=bias, padding=0)


In [29]:
torch.      

NameError: name 'window' is not defined